# Decorator Application: Single Dispatch Generic Functions

In [1]:
from html import escape

In [4]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li<{0}</li>'.format(html_escape(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(k, v)
             for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [5]:
print(html_str("""this is
a multi line string
with special characters 10 < 100"""))

this is<br/>
a multi line string<br/>
with special characters 10 &lt; 100


In [6]:
print(html_int(255))

255(<i>0xff</i>


In [7]:
print(html_escape(3+10j))

(3+10j)


In [8]:
from decimal import Decimal

In [10]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [11]:
htmlize(100)

'100(<i>0x64</i>'

In [12]:
htmlize("""Python
rocks!
""")

'Python<br/>\nrocks!<br/>\n'

In [13]:
print(htmlize([1,2,3]))

<ul>
<li<1</li>
<li<2</li>
<li<3</li>
</ul>


In [15]:
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li<Python
rocks! 0 &lt; 1
</li>
<li<(10, 20, 30)</li>
<li<100</li>
</ul>


In [18]:
# In Python, we can refernece a function (call a function) in the body of another function before that
# function has been defined, as long as the function has been defined by the time you call the function
def func1():
    func2()

In [19]:
def func2():
    print('func2')

In [20]:
func1()

func2


In [1]:
from decimal import Decimal
from html import escape

In [8]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg, set):
        return html_set(arg)
    else:
        return html_escape(arg)

In [14]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), htmlize(v))
             for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(s):
    return html_list(s)

In [15]:
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python<br/>
rocks! 0 &lt; 1<br/>
</li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>20(<i>0x14</i>)</li>
<li>30(<i>0x1e</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


In [22]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict
    }
    
    fn = registry.get(type(arg), registry[object])

    return fn(arg)

In [23]:
def single_dispatch(fn):
    registry = {}

    registry[object] = fn
    def inner(arg):
        return registry[object](arg)

    return inner

In [24]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [26]:
htmlize('1 < 100')

'1 &lt; 100'

In [29]:
def single_dispatch(fn):
    registry = {}

    registry[object] = fn
    registry[int] = lambda a: '{0}(<i>{1}</i>)'.format(a, str(hex(a)))
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n') 


    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)

    return inner

In [30]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [31]:
htmlize('1 < 100')

'1 &lt; 100'

In [32]:
htmlize(100)

'100(<i>0x64</i>)'

In [66]:
def single_dispatch(fn):
    registry = {}

    registry[object] = fn


    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner

    def dispatch(type_):
        return registry.get(type(type_), registry[object])

    decorated.register = register
    # decorated.registry = registry
    decorated.dispatch = dispatch

    return decorated

In [67]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [68]:
htmlize('1 < 100')

'1 &lt; 100'

In [69]:
htmlize.register

<function __main__.single_dispatch.<locals>.register(type_)>

In [70]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [71]:
htmlize(100)

'100(<i>0x64</i>)'

In [72]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [73]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [74]:
htmlize((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [76]:
htmlize.dispatch

<function __main__.single_dispatch.<locals>.dispatch(type_)>

In [77]:
htmlize(True)

'True'

In [78]:
from numbers import Integral

In [79]:
class Person:
    pass

In [80]:
class Student(Person):
    pass

In [81]:
p = Student()

In [82]:
type(p)

__main__.Student

In [83]:
isinstance(p, Student)

True

In [84]:
isinstance(p, Person)

True

In [85]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [86]:
@htmlize.register(Integral)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [88]:
isinstance(True, Integral)

True

In [89]:
htmlize(10)

'10'

In [90]:
@htmlize.register(int)
@htmlize.register(bool)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [91]:
htmlize(10)

'10(<i>0xa</i>)'

In [92]:
htmlize(True)

'True(<i>0x1</i>)'

In [93]:
from collections.abc import Sequence

In [94]:
isinstance([1, 2, 3], Sequence)

True

In [95]:
isinstance((1, 2, 3), Sequence)

True

In [97]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [98]:
type([1, 2, 3]) is Sequence

False

In [99]:
from functools import singledispatch

In [100]:
from numbers import Integral
from collections.abc import Sequence

In [101]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [103]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [104]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [105]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [106]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>})

In [107]:
htmlize.dispatch(int)

<function __main__.htmlize_integral_number(a)>

In [108]:
type(10)

int

In [109]:
isinstance(10, int)

True

In [110]:
isinstance(10, Integral)

True

In [111]:
isinstance(True, Integral)

True

In [113]:
htmlize.dispatch(bool)

<function __main__.htmlize_integral_number(a)>

In [114]:
htmlize(10)

'10(<i>0xa</i>)'

In [115]:
htmlize(True)

'True(<i>0x1</i>)'

In [116]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [117]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [118]:
htmlize((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [119]:
isinstance('python', Sequence)

True

In [120]:
htmlize('python')

RecursionError: maximum recursion depth exceeded in comparison

In [122]:
@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [123]:
htmlize('python 1 < 100')

'python 1 &lt; 100'

In [124]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [125]:
htmlize((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [127]:
@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return '({0})'.format(', '.join(items))

In [128]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(l)>,
              str: <function __main__.html_str(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [129]:
htmlize((1, 2, 3))

'(1, 2, 3)'

In [130]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [131]:
@htmlize.register(Integral)
def _(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

@htmlize.register(Sequence)
def _(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
             for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(Sequence)
def _(s):
    return html_escape(s).replace('\n', '</br>\n')

In [132]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__._(a)>,
              collections.abc.Sequence: <function __main__._(s)>})

In [133]:
htmlize(100)

'100(<i>0x64</i>)'

In [134]:
htmlize([1, 2, 3])

'[1, 2, 3]'

In [135]:
_

<function __main__._(s)>

In [136]:
id(htmlize.dispatch(Integral))

140368670203616

In [137]:
id(htmlize.dispatch(Sequence))

140368525901888

In [138]:
id(_)

140368525901888

In [139]:
a, _, c = 1, 2, 3

In [140]:
a

1

In [141]:
c

3

In [142]:
_

2

In [143]:
a, *_, c = 1, 2, 3, 4, 5

In [144]:
a

1

In [146]:
_

[2, 3, 4]

In [147]:
c

5